In [1]:
import time
import random
import pandas as pd
import numpy as np
from pathlib import Path
import os.path
import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_pickle('data/2020_stats_final.pkl')
df.columns = [column.capitalize() for column in df.columns]
df

,Jornada,Fecha,Local,Visitante,Goles local,Goles visitante,Link,Remates a puerta local,Remates a puerta visitante,Tiros local,Tiros visitante,Fueras de juego local,Fueras de juego visitante,Faltas local,Faltas visitante,Saques de esquina local,Saques de esquina visitante,Posesión local,Posesión visitante
0,1,2020-01-02,FBC Melgar,Universitario,1,2,https://www.ceroacero.es/match.php?id=7324623,7,2,20,7,1,1,15,21,8,3,59 %,41 %
1,1,2020-01-02,Alianza Lima,Alianza Huánuco,2,3,https://www.ceroacero.es/match.php?id=7324624,4,5,15,9,NaN,NaN,15,9,7,2,54 %,46 %
2,1,2020-01-02,Univ. Técnica,Sporting Cristal,2,1,https://www.ceroacero.es/match.php?id=7324625,5,6,19,14,4,0,21,8,2,5,39 %,61 %
3,1,2020-01-02,Sport Huancayo,Atlético Grau,1,0,https://www.ceroacero.es/match.php?id=7324626,6,3,24,4,3,2,9,8,13,4,65 %,35 %
4,1,2020-02-02,Univ. César Vallejo,Deportivo Municipal,1,1,https://www.ceroacero.es/match.php?id=7324627,6,1,18,5,1,4,13,18,13,1,75 %,25 %
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,19,2020-10-19,Ayacucho FC,FC Carlos Stein,3,0,https://www.ceroacero.es/match.php?id=7324816,7,3,16,5,4,0,14,13,4,2,56 %,44 %
186,19,2020-10-19,Deportivo Llacuabamba,Univ. César Vallejo,2,3,https://www.ceroacero.es/match.php?id=7324815,4,8,14,13,2,4,13,24,5,5,47 %,53 %
187,19,2020-10-19,Universitario,Cusco FC,3,2,https://www.ceroacero.es/match.php?id=7324820,4,8,10,23,1,1,15,18,5,4,56 %,44 %
188,19,2020-10-19,Atlético Grau,Univ. Técnica,0,0,https://www.ceroacero.es/match.php?id=7324818,3,1,10,7,NaN,NaN,10,27,9,4,61 %,39 %


In [3]:
def extract_stats(df):
    for jornada, local, visit, link in zip(df.Jornada, df.Local, df.Visitante, df.Link):
        
        dir = f"data/2020/Jornada {jornada}"
        Path(dir).mkdir(parents=True, exist_ok=True)

        if not os.path.exists(f'{dir}/{local} vs {visit}.xlsx'):

            print(jornada, link, local,'vs',visit)

            # Games Stats
            teams_stats = pd.read_html(link.replace('match','match_performance'))

            if len(teams_stats)<2:

                # Create a Pandas Excel writer using XlsxWriter as the engine.
                writer = pd.ExcelWriter(f'{dir}/NA {local} vs {visit}.xlsx', engine='xlsxwriter')

                writer.save()
                writer.close()

                time.sleep(random.randint(5, 10))
                continue

            # Create a Pandas Excel writer using XlsxWriter as the engine.
            writer = pd.ExcelWriter(f'{dir}/{local} vs {visit}.xlsx', engine='xlsxwriter')

            # Splitting Stats by Local - Visit
            local_stats = teams_stats[2].rename(columns={'Unnamed: 0':'Jugador'})
            visit_stats = teams_stats[3].rename(columns={'Unnamed: 0':'Jugador'})

            # Convert the dataframe to an XlsxWriter Excel object.
            local_stats.to_excel(writer, sheet_name=f'{local}', index=False)
            visit_stats.to_excel(writer, sheet_name=f'{visit}', index=False)

            # Close the Pandas Excel writer and output the Excel file.
            writer.save()
            writer.close()

            time.sleep(random.randint(20, 30))
        

In [4]:
extract_stats(df)

1 https://www.ceroacero.es/match.php?id=7324623 FBC Melgar vs Universitario
1 https://www.ceroacero.es/match.php?id=7324624 Alianza Lima vs Alianza Huánuco
1 https://www.ceroacero.es/match.php?id=7324625 Univ. Técnica vs Sporting Cristal
1 https://www.ceroacero.es/match.php?id=7324626 Sport Huancayo vs Atlético Grau
1 https://www.ceroacero.es/match.php?id=7324627 Univ. César Vallejo vs Deportivo Municipal
1 https://www.ceroacero.es/match.php?id=7324628 Univ. San Martín vs Ayacucho FC
1 https://www.ceroacero.es/match.php?id=7324629 Academia Cantolao vs Cienciano
1 https://www.ceroacero.es/match.php?id=7324630 FC Carlos Stein vs Carlos A. Mannucci
1 https://www.ceroacero.es/match.php?id=7324631 Sport Boys Callao vs Deportivo Llacuabamba
1 https://www.ceroacero.es/match.php?id=7324632 Cusco FC vs Binacional
2 https://www.ceroacero.es/match.php?id=7324633 Ayacucho FC vs Sport Boys Callao
2 https://www.ceroacero.es/match.php?id=7324634 Atlético Grau vs Univ. César Vallejo
2 https://www.cero